# Wirebond Test

## Prerequisite
You need to have Ansys installed locally (any version) - at the time of this notebook, Ansys is only supported in Windows.

## 1. Create the design in Metal
#### 1.1 Preload libraries and classes, then open Metal
Let's start by importing all the key libraries and classes, and opening Metal

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import qiskit_metal as metal
from qiskit_metal import designs, MetalGUI
from qiskit_metal import Dict, Headings

In [3]:
from qiskit_metal.renderers.renderer_ansys.ansys_renderer import QAnsysRenderer
QAnsysRenderer.default_options

{'Lj': '10nH',
 'Cj': 0,
 '_Rj': 0,
 'max_mesh_length_jj': '7um',
 'project_path': None,
 'project_name': None,
 'design_name': None,
 'ansys_file_extension': '.aedt',
 'x_buffer_width_mm': 0.2,
 'y_buffer_width_mm': 0.2,
 'plot_ansys_fields_options': {'name': 'NAME:Mag_E1',
  'UserSpecifyName': '0',
  'UserSpecifyFolder': '0',
  'QuantityName': 'Mag_E',
  'PlotFolder': 'E Field',
  'StreamlinePlot': 'False',
  'AdjacentSidePlot': 'False',
  'FullModelPlot': 'False',
  'IntrinsicVar': "Phase='0deg'",
  'PlotGeomInfo_0': '1',
  'PlotGeomInfo_1': 'Surface',
  'PlotGeomInfo_2': 'FacesList',
  'PlotGeomInfo_3': '1'}}

In [4]:
design = designs.DesignPlanar()
gui = MetalGUI(design)

In [5]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander

#### 1.2 Prepare a design consisting of 4 qubits and 4 CPWs
First we import the necessary components and change key global variables to the desired size.

In [6]:
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.interconnects.meandered import RouteMeander

In [7]:
design.variables['cpw_width'] = '15 um'
design.variables['cpw_gap'] = '9 um'

In [8]:
RouteMeander.get_template_options(design)

{'pin_inputs': {'start_pin': {'component': '', 'pin': ''},
  'end_pin': {'component': '', 'pin': ''}},
 'fillet': '0',
 'lead': {'start_straight': '0mm',
  'end_straight': '0mm',
  'start_jogged_extension': '',
  'end_jogged_extension': ''},
 'total_length': '7mm',
 'chip': 'main',
 'layer': '1',
 'trace_width': 'cpw_width',
 'meander': {'spacing': '200um', 'asymmetry': '0um'},
 'snap': 'true',
 'prevent_short_edges': 'true',
 'hfss_wire_bonds': False,
 'q3d_wire_bonds': False}

Let's create the 4-Qubit-4-CPWs design in one shot. Feel free to break the cell below into subcells if you would like to more closely follow the layout generation.

In [9]:
# Enable overwrite, so that we can re-un this cell multiple times without incurring object name collisions
design.overwrite_enabled = True

# We now define the same custom options for all the transmons.
# We only list the options we intend to modify with respect to the default values
options = Dict(
    pad_width = '425 um', 
    pocket_height = '650um',
    # Adding 3 connection_pads
    connection_pads=Dict(
        a = dict(loc_W=+1,loc_H=-1, pad_width='200um'),
        b = dict(loc_W=-1,loc_H=+1, pad_height='30um'),
        c = dict(loc_W=-1,loc_H=-1, pad_height='50um')
    )
)

# Create the 4 transmons
q1 = TransmonPocket(design, 'Q1', options = dict(
    pos_x='+2.42251mm', pos_y='+0.0mm', **options))
q2 = TransmonPocket(design, 'Q2', options = dict(
    pos_x='+0.0mm', pos_y='-0.95mm', orientation = '270', **options))
q3 = TransmonPocket(design, 'Q3', options = dict(
    pos_x='-2.42251mm', pos_y='+0.0mm', orientation = '180', **options))
q4 = TransmonPocket(design, 'Q4', options = dict(
    pos_x='+0.0mm', pos_y='+0.95mm', orientation = '90', **options))

# Now we do the same for the CPWs
options = Dict(
        lead=Dict(
            start_straight='0.2mm',
            end_straight='0.2mm'),
        trace_gap='9um',
        trace_width='15um')

# We define a handy method to shorten the number of line of code necessary to define the 4 CPW
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, asymmetry='0 um', flip=False, fillet='90um'):
    """Connect two pins with a CPW."""
    myoptions = Dict(hfss_wire_bonds=True,
        fillet=fillet,
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        total_length=length)
    myoptions.update(options)
    myoptions.meander.asymmetry = asymmetry
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

# Create the 4 CPWs
asym = 140
cpw1 = connect('cpw1', 'Q1', 'c', 'Q2', 'b', '5.6 mm', f'+{asym}um')
cpw2 = connect('cpw2', 'Q3', 'b', 'Q2', 'c', '5.7 mm', f'-{asym}um', flip=True)
cpw3 = connect('cpw3', 'Q3', 'c', 'Q4', 'b', '5.6 mm', f'+{asym}um')
cpw4 = connect('cpw4', 'Q1', 'b', 'Q4', 'c', '5.7 mm', f'-{asym}um', flip=True)

# Render the entire design to the Qiskit Metal GUI
gui.rebuild()
gui.autoscale()

## 2. Render into Ansys HFSS
The Ansys HFSS renderer was instantiated during the boot of Qiskit Metal. Let's create a short handle to refer to it in the rest of this notebook.

In [10]:
fourq_hfss = design.renderers.hfss

#### 2.1 Setup an Ansys project
To setup the project **manually**, follow these instructions:
1. Launch `ANSYS Electronics Desktop yyyy Rx` (from your Windows Start menu).
2. Create a new Ansys project by clicking on the `New` icon at the top left. (or open an existing project)

Alternatively, you can **automatically** set up the project by executing the following two cells. Make sure to wait after executing the first cell for Ansys to completely open. Only then execute the second cell.

Note about Ansys version: open_ansys() will look by default for the 2020 R2 version of Ansys. You can easily reroute it to your Ansys of choice by providing the name of the environment variable that contains the path (path_env) or the path itself (path)

In [ ]:
fourq_hfss.open_ansys()   # this opens Ansys 2020 R2 if present
# fourq_hfss.open_ansys(path_var='ANSYSEM_ROOT211')
# fourq_hfss.open_ansys(path='C:\Program Files\AnsysEM\AnsysEM20.2\Win64')
# fourq_hfss.open_ansys(path='../../../Program Files/AnsysEM/AnsysEM20.2/Win64')

NOTE: A new project should have automatically opened with the execution of the cell above. If not, uncomment the cell below and execute it. You can also load an existing project by passing the project information to the `connect_ansys()`.

In [ ]:
# fourq_hfss.new_ansys_project()

#### 2.2 Connect the Metal renderer with the Ansys project
Open either a new or existing design based on default options

In [11]:
fourq_hfss.connect_ansys()
# fourq_hfss.connect_ansys('C:\\project_path\\', 'Project1')  # Example of opening a saved project

INFO 02:39PM [connect_project]: Connecting to Ansys Desktop API...
INFO 02:39PM [load_ansys_project]: 	Opened Ansys App
INFO 02:39PM [load_ansys_project]: 	Opened Ansys Desktop v2020.2.0
INFO 02:39PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/ThomasMcConkey/Documents/Ansoft/
	Project:   Project2
INFO 02:39PM [connect_design]: 	Opened active design
	Design:    HFSSDesign1 [Solution type: DrivenModal]
WARNING 02:39PM [connect_setup]: 	No design setup detected.
WARNING 02:39PM [connect_setup]: 	Creating drivenmodal default setup.
INFO 02:39PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)
INFO 02:39PM [connect]: 	Connected to project "Project2" and design "HFSSDesign1" 😀 



You can also indicate, using the optional parameters, whether you intend to open and use a previously saved project. <br>
Make sure that the saved project contains at least one design, or the method will produce a warning

#### 2.3 Setup an Ansys HFSS design
You can either create a new design or select and use an old one.

To **create** a new design **manually**, go to the Ansys GUI and follow these instructions:
1. Select the project from the leftmost menu in the Ansys GUI.
2. Go into the menu `Project` and select `Insert HFSS Design`.
3. Change the HFSS design to either eigenmode or modal by right-clicking on the HFSSdesign1 that just got created inside your project (left panel) and then selecting: `Solution Type...`.

To **create** a new eigenmode design **automatically**, execute the following cell<br>
The design will be added to the project that was active when the command `fourq_hfss.connect_ansys()` was executed.
Note: If a design named `HFSSTransmonQubit` already exists in the project, a new design will be created, with the name suffixed with an incremental integer: `HFSSTransmonQubit1`, `HFSSTransmonQubit2`, etc.

In [ ]:
fourq_hfss.add_eigenmode_design("HFSSMetalEigenmode")

To **create** a new modal design **automatically**, execute the following cell instead

In [12]:
fourq_hfss.add_drivenmodal_design("HFSSMetalDrivenModal")

INFO 02:39PM [connect_design]: 	Opened active design
	Design:    HFSSMetalDrivenModal [Solution type: DrivenModal]
WARNING 02:39PM [connect_setup]: 	No design setup detected.
WARNING 02:39PM [connect_setup]: 	Creating drivenmodal default setup.
INFO 02:39PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssDMSetup'>)


In [21]:
fourq_hfss.modeler.draw_wirebond

<bound method HfssModeler.draw_wirebond of <pyEPR.ansys.HfssModeler object at 0x00000163276E9748>>

In pyEPR

def draw_wirebond(self,
                      pos,
                      ori,
                      width,
                      height='0.1mm',
                      z=0,
                      wire_diameter="0.02mm",
                      NumSides=6,
                      **kwargs):

def easy_wirebond(design, obj,
                  options = {},
                  name = None,
                  draw_hfss=True,
                  parent_obj = None):
    '''
    TODO: split into make and draw maybe add as funciton to cpw_ base object? that calls this
    Add wire bonds automatically to a CPW trace given a certain spacing
    Assumes zero elevation for chip. Not flip chip
    Units: User units
    OLD raw use:
        from qiskit_metal.draw_cpw import *
        name = 'cpw_Q1_bus_Q2_Q2_bus_Q1'
        easy_wirebond(design, objects[name], 'Bond_'+name, Dict(
            start=0, stop=-1, step=2, threshold='0.2mm'))
    TEST:
        options = objects['cpw_Q1_bus_Q2_Q2_bus_Q1']['options_hfss']
        cpw_line = objects['cpw_Q1_bus_Q2_Q2_bus_Q1']['objects']['cpw_line']#.objects.cpw_line
        points_meander = np.array(cpw_line.coords)
    '''

    options = {**DEFAULT_OPTIONS['easy_wirebond'], **options} # overwritten for Metal Object

    ############################################
    # Get points of cpw and handle Metal Object
    from .objects.base_objects.Metal_Utility import is_metal_object

    if is_metal_object(obj):

        if parent_obj is None:
            parent_obj = obj

        if name is None:
            name ='Bond_' + obj.name

        options = {**DEFAULT_OPTIONS['easy_wirebond'],
                   **obj.options.easy_wirebond, # if there are already wirebond options
                   **options}

        if 'cpw_line' in obj.objects:
            points_meander = np.array(obj.objects.cpw_line.coords)
        else:
            raise Exception('Object {obj} does not have obj.objects.cpw_line')

    ############################################
    # Handle non metal obejcts (yes, too many here, not all needed)
    elif isinstance(obj, Dict):
        if ('objects' in obj):
            if 'cpw_line' in obj.objects:
                points_meander = np.array(obj.objects.cpw_line.coords)
            else:
                raise Exception('Unkown object {obj} does not have obj.objects.cpw_line')
        else:
            raise Exception('Unkown object {obj}: does not have obj.objects')
    elif isinstance(obj, str):
        points_meander = np.array(unparse_units(design.track_objs[options['category']][obj]['points']))
        #DEPRICATED here
    elif isinstance(obj, shapely.geometry.LineString):
        points_meander = np.array(obj.coords)
    elif isinstance(obj, np.ndarray) or isinstance(obj, list):
        points_meander = np.array(obj)
    else:
        points_meander = np.array(obj)
        logger.error(
            'ERROR: UNKOWN INPUT OBJECT! Cannot convert to points!  Trying to procees')
        raise Exception('See logger error.')

    ############################################
    # Core part of procedure
    #
    # Figure out new control points and draw
    # Place once per segment;  iterate each path segement

    start, stop, step = [options[n.strip()] for n in ('start, stop, step'.split(','))]
    w, ofst, th, height = parse_options_user(options, ['w', 'offset', 'threshold', 'height'])

    wirebond_names = []
    shapes = {}
    for i in range(start, len(points_meander) + stop, step):
        p1, p2 = map(array, points_meander[i:i+2])
        vec_D, vec_d, vec_n = get_vec_unit_norm([p1, p2])
        #print(p1, vec_n, norm(vec_D), '\n ',p1,p2, vec_n ) #draw_objs([Point(p1),Point(p2)])

        if (norm(vec_D) > th) and (norm(vec_D)/2. >  ofst): # if the segment is longer than thresohld place a bond
           # make sure wirbondf doesn stick out
            pos, ori = [(p1+p2)/2., vec_n]
            pos -= ofst*vec_d   # w*vec_n/2.

            p = np.array(pos)
            shapes[str(i)] = dict(center=Point(pos),
                             bond = LineString([p-ori*w/2, p+ori*w/2]))
            #draw_objs(shapes[i]) # draw shapely
            if draw_hfss:
                _, oModeler = design.get_modeler() ###
                wirebond_names += [
                    oModeler.draw_wirebond(parse_units(pos), ori, parse_units(w),
                                        height=parse_units(height),
                                        name=name+f'_{i}',
                                        material='pec',
                                        solve_inside = False)]

    ############################################
    #
    # Save new objects to parent object
    if not (parent_obj is None):

        if is_metal_object(obj):
            obj = parent_obj
            obj.objects.wirebonds = shapes
            obj.hfss_objects = wirebond_names
            obj.options.easy_wirebond = options
        else:
            if not ('hfss_objects' in parent_obj):
                parent_obj['hfss_objects'] = Dict()

            parent_obj['hfss_objects']['wirebonds'] = Dict(
                options_easy_wirebond = options,
                wirebond_names = wirebond_names
            )
            parent_obj['objects_wirebond'] = shapes

In [33]:
design.chips.main

{'material': 'silicon',
 'layer_start': '0',
 'layer_end': '2048',
 'size': {'center_x': '0.0mm',
  'center_y': '0.0mm',
  'center_z': '0.0mm',
  'size_x': '9mm',
  'size_y': '6mm',
  'size_z': '-750um',
  'sample_holder_top': '890um',
  'sample_holder_bottom': '1650um'}}

In [14]:
type(design.qgeometry.tables['path'])#['hfss_wire_bonds']

geopandas.geodataframe.GeoDataFrame

In [15]:
design._components[5].name

'cpw1'

In [16]:
wb_table=design.qgeometry.tables['path'].loc[design.qgeometry.tables['path']['hfss_wire_bonds'] == True]
wb_table2 = wb_table.loc[wb_table['subtract']==True]
wb_table2

,component,name,geometry,layer,subtract,helper,chip,width,fillet,hfss_wire_bonds,q3d_wire_bonds,gds_wire_bonds
25,5,cut,"LINESTRING (1.99751 -0.19750, 1.79751 -0.19750...",1,True,False,main,0.033,0.09,True,False,NaN
27,6,cut,"LINESTRING (-1.99751 -0.19750, -1.79751 -0.197...",1,True,False,main,0.033,0.09,True,False,NaN
29,7,cut,"LINESTRING (-1.99751 0.19750, -1.79751 0.19750...",1,True,False,main,0.033,0.09,True,False,NaN
31,8,cut,"LINESTRING (1.99751 0.19750, 1.79751 0.19750, ...",1,True,False,main,0.033,0.09,True,False,NaN


In [20]:
from scipy.spatial import distance
import numpy as np

In [37]:
geom=wb_table2.iloc[0]['geometry']
distance.euclidean(geom.coords[:][1], geom.coords[:][2])
#wb_table2['component'] == 5
distance.euclidean(geom.coords[:][0], geom.coords[:][1])

0.19999999999999996

In [28]:
np.asarray(geom.coords[:][0])

array([ 1.99751, -0.1975 ])

In [29]:
geom.coords[:][0]

(1.99751, -0.1975)

In [31]:

wb_pos = tuple(map(lambda i, j: i+(j-i)/2, geom.coords[:][0], geom.coords[:][1]))
i_p = np.asarray(geom.coords[:][0])
j_p = np.asarray(geom.coords[:][1])
i_p + (j_p - i_p)/2

array([ 1.89751, -0.1975 ])

In [32]:
wb_pos

(1.89751, -0.1975)

In [120]:
numpy.linalg.norm((1,1))

1.4142135623730951

In [43]:
norm_z = np.array([0,0,1])
np.cross(norm_z,np.array([1,0]))[:2]
a = 1
a+=1
a

2

In [50]:
wb_pos = (j_p - i_p)/2
wb_perp = np.cross(norm_z, wb_pos/np.linalg.norm(wb_pos))[:2]
wb_perp
wb_pos += i_p 
wb_pos

array([ 1.89751, -0.1975 ])

In [44]:
norm_z = np.array([0,0,1])

for index, i_p in enumerate(geom.coords[:-1],start=0):
    j_p = np.asarray(geom.coords[:][index+1])
    if distance.euclidean(i_p, j_p)>wb_threshold:
        #Modification for multiple wirebonds put stretch of cpw would be made here
        wb_pos = (j_p - i_p)/2
        wb_perp = np.cross(norm_z, wb_pos/np.linalg.norm(wb_pos))[:2]
        #wb_pos += i_p 
        
    

NameError: name 'wb_threshold' is not defined

In [75]:
#from pyEPR.ansys import parse_units
#fourq_hfss.modeler.draw_wirebond(parse_units(('1.795mm','0.4mm')), (-1,0), parse_units('0.06mm'))
parse_units((1.795,0.4))

[0.001795, 0.0004]

In [63]:
%debug

> c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\site-packages\pyepr\ansys.py(2194)draw_wirebond()
   2192         p = np.array(pos)
   2193         o = np.array(ori)
-> 2194         pad1 = p - o * width / 2.
   2195         name = self._modeler.CreateBondwire([
   2196             "NAME:BondwireParameters", "WireType:=", "Low", "WireDiameter:=",



ipdb>  p


*** SyntaxError: unexpected EOF while parsing


ipdb>  pos


(1.795, 0.4)


...

ipdb>  p


*** SyntaxError: unexpected EOF while parsing


ipdb>  p = np.array(pos)


*** SyntaxError: invalid syntax


ipdb>  exit


In [94]:


wb_offset = '50um'
wb_threshold = '400um'
norm_z = np.array([0,0,1])

wb_table=design.qgeometry.tables['path'].loc[design.qgeometry.tables['path']['hfss_wire_bonds'] == True]
wb_table2 = wb_table.loc[wb_table['subtract']==True]
wb_table2

for index, row in wb_table2.iterrows():
    geom = row['geometry']
    width = row['width']
    name = design._components[row['component']].name
    for index, i_p in enumerate(geom.coords[:-1],start=0):
        j_p = np.asarray(geom.coords[:][index+1])
        if parse_units(distance.euclidean(i_p, j_p))>parse_units(wb_threshold):
            #Modification for multiple wirebonds put stretch of cpw would be made here
            wb_pos = (j_p - i_p)/2
            wb_vec = wb_pos/np.linalg.norm(wb_pos)
            wb_perp = np.cross(norm_z, wb_vec)[:2]
            wb_pos = parse_units(wb_pos+i_p) + (wb_vec * parse_units(wb_offset))
            #wb_pos = wb_pos + wb_perp * width
            fourq_hfss.modeler.draw_wirebond(wb_pos, wb_perp, parse_units(width*3))
            
# 
#
# def draw_wirebond(self,
#                      pos,
#                      ori,
#                      width,
#                      height='0.1mm',
#                      z=0,
#                      wire_diameter="0.02mm",
#                      NumSides=6,
#                      **kwargs):

In [87]:
fourq_hfss.modeler.draw_wirebond(parse_units([ 1.59751, -0.3375 ]), [-1.,  0.], parse_units(width*3))

'Bondwire1'

In [85]:
%debug

> <ipython-input-84-e29154a9cce3>(22)<module>()
     20             wb_pos += (i_p + wb_vec * parse_units(wb_offset))
     21             #wb_pos = wb_pos + wb_perp * width
---> 22             a = 1/0
     23             fourq_hfss.modeler.draw_wirebond(parse_units(wb_pos), parse_units(wb_perp), parse_units(width*3))
     24 



ipdb>  wb_pos


array([ 1.59751, -0.3375 ])


ipdb>  wb_perp


array([-1.,  0.])


ipdb>  parse_units(width*3)


9.900000000000001e-05


ipdb>  exit


In [48]:
design.qgeometry.tables['path']['hfss_wire_bonds'] == True

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24     True
25     True
26     True
27     True
28     True
29     True
30     True
31     True
Name: hfss_wire_bonds, dtype: bool

To **select** an existing design, you can follow one of three ways:
* re-running the `fourq_hfss.connect_ansys(*with parameters*)`, this time specifying which design to connect to (see section 2.2)
* manually activating the design from the Ansys GUI. You will find the list of designs in the leftmost panel, and you can activate them with a double click. After this, re-run the `fourq_hfss.connect_ansys()` *without parameters*.
* use methods: activate_eigenmode_design() or activate_drivenmodal_design().  If the design name exists, it will be added, but no integer will be added to the suffix. If the design name does not exist, then it will be added to the project.

Let's illustrate the **automated** method to select an existing design. The following two cells will switch between the eigenmode and the modal designs we just created in the last two cells above, or it will create them if you did not run the previous cells:

In [ ]:
fourq_hfss.activate_eigenmode_design("HFSSMetalEigenmode")

In [ ]:
fourq_hfss.activate_drivenmodal_design("HFSSMetalDrivenModal")

#### 2.4 Render some component from the Metal design

Find below several rendering examples. You can choose to only execute one of them if you are just browsing this notebook.

Notice how we explicitly clear the design before re-rendering. Indeed `render_design()` only adds shapes to the Ansys design. Re-rendering the same shapes will cause violations.

In [51]:
fourq_hfss.render_design([], [])  # entire Metal design.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')])  # single qubit with 2 endcaps.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')])  # 2 qubits and 2 endcaps, one per qubit.

For Driven-Modal analysis, we can also add terminations. In the example below we render 1 qubit with 1 endcap and 1 port with a 70 Ohm termination.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design(['Q2'], [('Q2', 'a')], [('Q2', 'b', '70')])

In the previous examples, rendering area dimenstions is determined by the size of the selected geometries, with some buffer.

For a more accurate control of the chip size, you need to disable the buffering as below. This will use `design._chips['main']['size']` to determine the rendering area dimensions.

In [ ]:
fourq_hfss.clean_active_design()
fourq_hfss.render_design([], [], box_plus_buffer=False)

You can also modify the chip size directly by updating `design._chips['main']['size']`. Example below.

NOTE: we purposfully make the chip size smaller than the size of the geometry. This will cause a warning to show which will need to be fixed by the user intending to conduct a valid analysis.

In [ ]:
fourq_hfss.clean_active_design()
design._chips['main']['size']['size_x'] = '4mm'
fourq_hfss.render_design([], [], box_plus_buffer=False)
#    Return back to original size, for the remainder of the notebook
design._chips['main']['size']['size_x'] = '6mm'

#### 2.5 Create of select a setup
This section is only needed if you plan to execute an analysis. This topic will be covered in the tutorials in chapter 6.

To **select** a setup for the active design, you can use `activate_drivenmodal_setup()` or `activate_eigenmode_setup()`. If the setup exists, the QRenderer will reference the setup, otherwise, will make a new setup with the name give.  If no name given, the default name of "Setup" will be used. Below the example for eigenmode.

In [ ]:
fourq_hfss.activate_eigenmode_setup('SetupNEW')

**Finally** disconnect the Metal renderer from the Ansys session.

NOTE: This is needed every time before re-connecting. If you do not disconnect explicitly, you might not be able to close the Ansys GUI later.

In [25]:
fourq_hfss.disconnect_ansys()

## 3. Render into Ansys Q3D
The Ansys Q3D renderer was instantiated during the boot of Qiskit Metal. Let's create a short handle to refer to it in the rest of this notebook.

In [ ]:
fourq_q3d = design.renderers.q3d

#### 3.1 Setup an Ansys project
Skip this section if ansys is already open. See section 2.1 in this notebook for additional details and options.

In [ ]:
fourq_q3d.open_ansys()

#### 3.2 Connect the Q3D renderer with the Ansys project
Open either a new or existing design based on default options. See section 2.2 in this notebook for details and options.

In [ ]:
fourq_q3d.connect_ansys()
# fourq_q3d.connect_ansys('C:\\project_path\\', 'Project1')  # Example of opening a saved project

#### 3.3 Setup an Ansys Q3D design
You can either create a new design or select and use an old one.

To **create** a new design **manually**, go to the Ansys GUI and follow these instructions:
1. Select the project from the leftmost menu in the Ansys GUI.
2. Go into the menu `Project` and select `Insert Q3D Extractor Design`.

To **create** a new Q3D design **automatically**, execute the following cell.<br>
Note: If a design named `Q3dMetalDesign` already exists in the project, a new design will be created, with the name suffixed with an incremental integer: `Q3dMetalDesign1`, `Q3dMetalDesign2`, etc.

In [ ]:
fourq_q3d.add_q3d_design("Q3dMetalDesign")

To **select** an existing design, means to activate an Ansys design and connect to it. You can follow one of three ways:
* re-running the `fourq_q3d.connect_ansys(*with parameters*)`, this time specifying which design to connect to (see section 2.3)
* manually activating the design from the Ansys GUI. You will find the list of designs in the leftmost panel, and you can activate them with a double click. After this, re-run the `fourq_q3d.connect_ansys()` *without parameters*.
* use method: activate_q3d_design().  If the design name exists, it will be added, but no integer will be added to the suffix. If the design name does not exist, then it will be added to the project.

Let's illustrate the **automated** method to select an existing design. The first of the two cells below will create a new design because the name does not corrispond to any design in the project. The second cell will return to the previously created design.

In [ ]:
fourq_q3d.activate_q3d_design("Q3dMetalDesignNEW")

In [ ]:
fourq_q3d.activate_q3d_design("Q3dMetalDesign")

#### 3.4 Render some component from the Metal design

Find below several rendering examples. You can choose to only execute one of them if you are just browsing this notebook.

Notice how we explicitly clear the design before re-rendering. Indeed `render_design()` only adds shapes to the Ansys design. Re-rendering the same shapes will cause violations.

In [ ]:
fourq_q3d.render_design([], [])  # entire Metal design.

In [ ]:
fourq_q3d.clean_active_design()
fourq_q3d.render_design(['Q1'], [('Q1', 'b'), ('Q1', 'c')])  # single qubit with 2 endcaps.

In [ ]:
fourq_q3d.clean_active_design()
fourq_q3d.render_design(['Q1', 'cpw1', 'Q2'], [('Q1', 'b'), ('Q2', 'c')])  # 2 qubits and 2 endcaps, one per qubit.

In the previous examples, rendering area dimenstions is determined by the size of the selected geometries, with some padding.

For a more accurate control of the chip size, you need to disable the buffering as below. This will use `design._chips['main']['size']` to determine the rendering area dimensions.

In [ ]:
fourq_q3d.clean_active_design()
fourq_q3d.render_design([], [], box_plus_buffer=False)

You can also modify the chip size directly by updating `design._chips['main']['size']`. Example below:

In [ ]:
fourq_q3d.clean_active_design()
design._chips['main']['size']['size_y'] = '4mm'
fourq_q3d.render_design([], [], box_plus_buffer=False)
#    Return back to original size, for the remainder of the notebook
design._chips['main']['size']['size_y'] = '6mm'

#### 3.5 Create of select a setup
This section is only needed if you plan to execute an analysis. This topic will be covered in the tutorials in chapter 6.

To **select** a setup for the active design, you can use `activate_q3d_setup()`. If the setup exists, the QRenderer will reference the setup, otherwise, will make a new setup with the name give.  If no name given, the default name of "Setup" will be used.

In [ ]:
fourq_q3d.activate_q3d_setup("SetupNEW")

**Finally** disconnect the Metal renderer from the Ansys session. You will not be able to close Ansys without executing this.

In [24]:
fourq_q3d.disconnect_ansys()

[autoreload of qiskit_metal.renderers.renderer_ansys.ansys_renderer failed: Traceback (most recent call last):
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\site-packages\IPython\extensions\autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\site-packages\IPython\extensions\autoreload.py", line 394, in superreload
    module = reload(module)
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\imp.py", line 314, in reload
    return importlib.reload(module)
  File "c:\users\thomasmcconkey\anaconda3\envs\q_metal2\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 724, in exec_module
  File "<frozen importlib._bootstrap_external>", line 860, in get_code
  File "<frozen importlib._bootstrap_external>", line 791, in source_to_code
  File "<

NameError: name 'fourq_q3d' is not defined

If you want to close the Metal GUI, uncomment the following cell

In [ ]:
# gui.main_window.close()

## References - Miscellaneous pyEPR/Ansys commands
The following commands are for reference only to better understand how the backend code works. They're not meant to be run directly in this notebook as part of the tutorial.

import pyEPR as epr

Connect to Ansys directly from notebook:

pinfo = epr.ProjectInfo(project_path = None, 
                        project_name = None,
                        design_name  = None)
modeler = pinfo.design.modeler

Access methods within HfssDesign class in pyEPR:

epr.ansys.HfssDesign.create_dm_setup
epr.ansys.HfssDesign.create_q3d_setup

Get project and design names:

pinfo.project_name
design._design.GetName()

Filter qgeometry table:

full_table = design.qgeometry.tables['poly']
mask = full_table['subtract'] == False
table = full_table[mask]

Draw centered rectangles:

bigsquare = modeler.draw_rect_center([0, 0, 0], x_size=8, y_size=8, name='bigsquare')
topright = modeler.draw_rect_center([2, 2, 0], x_size=2, y_size=2, name='topright')

Subtracting shapes:

modeler.subtract('bigsquare', ['topright'])

Draw centered box:

modeler.draw_box_center([0, 0, 0], [1, 2, 3])

Draw closed polygon:

trianglepts = [[-1, 5, 0], [1, 5, 0], [0, 7, 0]]
modeler.draw_polyline(trianglepts, closed=True)

Draw polyline:

smallpts = [[2.85, 0, 0], [3.15, 0, 0]]
modeler.draw_polyline(smallpts, closed=False)

Sweep one polyline with another:

modeler._sweep_along_path('Polyline8', 'Polyline7')